In [1]:
import json
import numpy as np
import math


In [2]:
with open('flaregeometry.oofem','r') as json_file:
    flare = json.load(json_file)
with open('model.dsgn') as wind_file:
    wind_data = json.load(wind_file)
    
mesh_level = {'refine_level':2,'second_order':2, 'min_ele': 0.5}

In [3]:
class flaregeo(object):
    
    
    def __init__(self):
        self.section_num = {}
        for i in flare['BeamElements']:
            if i['Section'] not in self.section_num:
                self.section_num[i['Section']] = [i['Label']]
            else:
                self.section_num[i['Section']].append(i['Label'])
        self.radius = [wind_data['elementArray'][0]['Af']/2]
        for i in wind_data['elementArray']:
            self.radius.append(i['Af']/2)
        self.thicktype = {}
        for i in flare['BeamSections']:
            if i['Label'] not in self.thicktype:
                self.thicktype[i['Label']] = i['Area']/3.14
        
        self.section_coord = []
        for i in flare['Nodes']:
            self.section_coord.append(i['Coords'][2])
            
### flare geometry ##        
    def flare_geo(self):
        gmshfile = open('flare.geo','w')
        circlenum=1
        for i in range(1,len(self.section_coord)+1):
            
            gmshfile.write('Point(%d) = {0,0,%d,1};\n'%(5*i-4,self.section_coord[i-1])+\
                          'Point(%d) = {%d,0,%d,1};\n'%(5*i-3,self.radius[i-1],self.section_coord[i-1])+\
                          'Point(%d) = {0,%d,%d,1};\n'%(5*i-2,self.radius[i-1],self.section_coord[i-1])+\
                          'Point(%d) = {-%d,0,%d,1};\n'%(5*i-1,self.radius[i-1],self.section_coord[i-1])+\
                          'Point(%d) = {0,-%d,%d,1};\n'%(5*i,self.radius[i-1],self.section_coord[i-1]))
            gmshfile.write('Circle(%d) = {%d, %d, %d};\n'%(circlenum,5*i-3,5*i-4,5*i-2)+\
                          'Circle(%d) = {%d, %d, %d};\n'%(circlenum+1,5*i-2,5*i-4,5*i-1)+\
                          'Circle(%d) = {%d, %d, %d};\n'%(circlenum+2,5*i-1,5*i-4,5*i)+\
                          'Circle(%d) = {%d, %d, %d};\n'%(circlenum+3,5*i,5*i-4,5*i-3))
            circlenum +=4
        for i in range(1,len(self.section_coord)):
            linenum = 4*len(self.section_coord)
            gmshfile.write('Line(%d) = {%d, %d};\n'%(linenum+4*i-3,5*i-3,5*i+2)+\
                          'Line(%d) = {%d, %d};\n'%(linenum+4*i-2,5*i-2,5*i+3)+\
                          'Line(%d) = {%d, %d};\n'%(linenum+4*i-1,5*i-1,5*i+4)+\
                          'Line(%d) = {%d, %d};\n'%(linenum+4*i,5*i,5*i+5))
            
        for i in range(1,len(self.section_coord)):
            linenum = 4*len(self.section_coord)
            gmshfile.write('Line Loop(%d) = {%d, %d, %d, %d};\n'%(4*i-3,4*i-3,linenum+4*i-2,-(4*i+1),-(linenum+4*i-3))+\
                           'Surface(%d) = {%d};\n'%(4*i-3,4*i-3)+\
                           'Line Loop(%d) = {%d, %d, %d, %d};\n'%(4*i-2,4*i-2,linenum+4*i-1,-(4*i+2),-(linenum+4*i-2))+\
                           'Surface(%d) = {%d};\n'%(4*i-2,4*i-2)+\
                           'Line Loop(%d) = {%d, %d, %d, %d};\n'%(4*i-1,4*i-1,linenum+4*i,-(4*i+3),-(linenum+4*i-1))+\
                           'Surface(%d) = {%d};\n'%(4*i-1,4*i-1)+\
                           'Line Loop(%d) = {%d, %d, %d, %d};\n'%(4*i,4*i,linenum+4*i-3,-(4*i+4),-(linenum+4*i))+\
                           'Surface(%d) = {%d};\n'%(4*i,4*i))
            
        ### Physical groups ####    
        for i in self.section_num:
            surface_num =[]
            for j in self.section_num[i]:
                surface_num.extend((4*j-3,4*j-2,4*j-1,4*j))
            surface_str = ", ".join(map(str,surface_num))
            gmshfile.write('Physical Surface("section%s") = {%s};\n'%(str(i),surface_str))
            
        ### wind surface ###
        
        for i in range(1,len(flare['BeamElements'])+1):
            gmshfile.write('Physical Surface("windsurf%s") = {%s};\n'%(str(i),str(i)))
        gmshfile.close()
        return 0
    def mesh_refine(self):
        gmshfile = open('flare.geo','a')
        """
        Mesh option:
        1=MeshAdapt, 2=Automatic, 5=Delaunay, 6=Frontal, 7=BAMG, 8=DelQuad

        """
        gmshfile.write('Mesh.SurfaceFaces = 1;\n'+\
                'Mesh.VolumeEdges = 0;\n'+\
                'Mesh.ElementOrder = %d;\n'%mesh_level['second_order'])
            
        if(mesh_level['refine_level'] == 1):
            gmshfile.write('Mesh.OptimizeNetgen = 1;\n'+\
            'Mesh.Algorithm = 2;\n'+\
            'Mesh.Smoothing = 30;\n'+\
            'Mesh.CharacteristicLengthFromCurvature = 0;\n'+\
            'Mesh.CharacteristicLengthFromPoints = 1;\n'+\
            'Mesh.CharacteristicLengthFactor =2;\n'+\
            'Mesh.CharacteristicLengthMax = %.3f;\n'%mesh_level['min_ele'])
        elif (mesh_level['refine_level'] ==2):
             gmshfile.write('Mesh.OptimizeNetgen = 1;\n'+\
            'Mesh.Algorithm = 2;\n'+\
            'Mesh.Smoothing = 50;\n'+\
            'Mesh.CharacteristicLengthFromCurvature = 0;\n'+\
            'Mesh.CharacteristicLengthFromPoints = 1;\n'+\
            'Mesh.CharacteristicLengthFactor =1;\n'+\
            'Mesh.CharacteristicLengthMax = %.3f;\n'%mesh_level['min_ele'])
        else:
            assert(mesh_level['refine_level'] == 0)
            gmshfile.write('Mesh.OptimizeNetgen = 1;\n'+\
            'Mesh.Algorithm = 2;\n'+\
            'Mesh.Smoothing = 50;\n'+\
            'Mesh.CharacteristicLengthFromCurvature = 1;\n'+\
            'Mesh.CharacteristicLengthFromPoints = 0;\n'+\
            'Mesh.CharacteristicLengthFactor =2;\n'+\
            'Mesh.CharacteristicLengthMax = %.3f;\n'%mesh_level['min_ele'])
        gmshfile.write('Mesh 2;\n'+\
                       'Print.Background = 1;\n'+\
                       'BoundingBox;\n'+\
                       'Print "ScreenMesh.png";\n')
        gmshfile.write('Mesh.SaveAll = 1;\n'+\
                       'Mesh.SaveGroupsOfNodes = 1;\n'+\
                       'Save "flare_gmsh_test.inp";\n'+\
                       'Save "flare_gmsh_test.stl";\n'+\
                       'Exit;\n')
        gmshfile.close()
        return
        
    
        
    
        

In [4]:
Atest =flaregeo()
Atest.flare_geo()
Atest.section_num

{1: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 2: [16, 17],
 3: [18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45],
 4: [46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65],
 5: [66, 67, 68, 69, 70]}

In [75]:
test=[0,10,20,30];diameter={1:20,2:10};section_num=[1,1,2,2]

In [91]:
    def flare_geo(test,diameter,section_num):
        gmshfile = open('test.geo','w')
        circlenum=1
        for i in range(1,len(test)+1):
            gmshfile.write('Point(%d) = {0,0,%d,1};\n'%(5*i-4,test[i-1])+\
                          'Point(%d) = {%d,0,%d,1};\n'%(5*i-3,diameter[section_num[i-1]],test[i-1])+\
                          'Point(%d) = {0,%d,%d,1};\n'%(5*i-2,diameter[section_num[i-1]],test[i-1])+\
                          'Point(%d) = {-%d,0,%d,1};\n'%(5*i-1,diameter[section_num[i-1]],test[i-1])+\
                          'Point(%d) = {0,-%d,%d,1};\n'%(5*i,diameter[section_num[i-1]],test[i-1]))
            gmshfile.write('Circle(%d) = {%d, %d, %d};\n'%(circlenum,5*i-3,5*i-4,5*i-2)+\
                          'Circle(%d) = {%d, %d, %d};\n'%(circlenum+1,5*i-2,5*i-4,5*i-1)+\
                          'Circle(%d) = {%d, %d, %d};\n'%(circlenum+2,5*i-1,5*i-4,5*i)+\
                          'Circle(%d) = {%d, %d, %d};\n'%(circlenum+3,5*i,5*i-4,5*i-3))
            circlenum +=4
        for i in range(1,len(test)):
            linenum = 4*len(test)
            gmshfile.write('Line(%d) = {%d, %d};\n'%(linenum+4*i-3,5*i-3,5*i+2)+\
                          'Line(%d) = {%d, %d};\n'%(linenum+4*i-2,5*i-2,5*i+3)+\
                          'Line(%d) = {%d, %d};\n'%(linenum+4*i-1,5*i-1,5*i+4)+\
                          'Line(%d) = {%d, %d};\n'%(linenum+4*i,5*i,5*i+5))
            
        for i in range(1,len(test)):
            linenum = 4*len(test)
            gmshfile.write('Line Loop(%d) = {%d, %d, %d, %d};\n'%(4*i-3,4*i-3,linenum+4*i-2,-(4*i+1),-(linenum+4*i-3))+\
                           'Surface(%d) = {%d};\n'%(4*i-3,4*i-3)+\
                           'Line Loop(%d) = {%d, %d, %d, %d};\n'%(4*i-2,4*i-2,linenum+4*i-1,-(4*i+2),-(linenum+4*i-2))+\
                           'Surface(%d) = {%d};\n'%(4*i-2,4*i-2)+\
                           'Line Loop(%d) = {%d, %d, %d, %d};\n'%(4*i-1,4*i-1,linenum+4*i,-(4*i+3),-(linenum+4*i-1))+\
                           'Surface(%d) = {%d};\n'%(4*i-1,4*i-1)+\
                           'Line Loop(%d) = {%d, %d, %d, %d};\n'%(4*i,4*i,linenum+4*i-3,-(4*i+4),-(linenum+4*i))+\
                           'Surface(%d) = {%d};\n'%(4*i,4*i))

        return 0
    
    

In [92]:
flare_geo(test,diameter,section_num)

0

In [7]:
flare.keys()

[u'StructuralTemperatureLoad',
 u'ConstantElementLoad',
 u'BeamSections',
 u'Analysis',
 u'DisplacementBoundaryConditions',
 u'Nodes',
 u'DeadWeightLoad',
 u'BeamElements']

In [85]:
math.sqrt(flare['BeamSections'][1]['Area']/3.14)

225.44570631876613

In [23]:
flare['DeadWeightLoad']

[{u'Components': [0, 0, -10, 0, 0, 0], u'Label': u'DeadWeight'}]

In [32]:
flare['BeamSections'][0]['MomentOfInertia_z']/flare['BeamSections'][0]['Area']

1510377.7074285715

In [34]:
265719524548/175929.1886010284*2

3020755.4148458876

In [35]:
flare['BeamElements'][0]

{u'BodyLoads': [u'DeadWeight'],
 u'BoundaryLoads': [u'Windload00flare01'],
 u'Label': 1,
 u'Nodes': [1, 2],
 u'Section': 1,
 u'Zaxis': [0, 1, 0]}

In [37]:
flare['BeamSections'][0]["Area"]/3.14/3500

16.008115432304677

In [25]:
a = [1,2,3,4,5,6,7]

In [28]:
if('8' in [i] for i in a):
    print "there"

there


In [31]:
if('8' in [1] ):
    print "here"

In [39]:
flare.keys()

[u'StructuralTemperatureLoad',
 u'ConstantElementLoad',
 u'BeamSections',
 u'Analysis',
 u'DisplacementBoundaryConditions',
 u'Nodes',
 u'DeadWeightLoad',
 u'BeamElements']

In [43]:
flare['StructuralTemperatureLoad'][0]['Components']

[140, 0, 0]

In [17]:
wind_data['elementArray']


[{u'Af': 3500,
  u'F': 4862.788629347254,
  u'Kz': 0.8070126781242019,
  u'Qz': 2335.072571115128,
  u'Qzf': 1634.5507997805896},
 {u'Af': 3500,
  u'F': 4862.788629347254,
  u'Kz': 0.8070126781242019,
  u'Qz': 2335.072571115128,
  u'Qzf': 1634.5507997805896},
 {u'Af': 3500,
  u'F': 4862.788629347254,
  u'Kz': 0.8070126781242019,
  u'Qz': 2335.072571115128,
  u'Qzf': 1634.5507997805896},
 {u'Af': 3500,
  u'F': 4862.788629347254,
  u'Kz': 0.8070126781242019,
  u'Qz': 2335.072571115128,
  u'Qzf': 1634.5507997805896},
 {u'Af': 3500,
  u'F': 3552.7531569035386,
  u'Kz': 0.5896034268410714,
  u'Qz': 1706.003916880451,
  u'Qzf': 1194.2027418163157},
 {u'Af': 3500,
  u'F': 3742.727835842952,
  u'Kz': 0.6211310103147666,
  u'Qz': 1797.2282525056194,
  u'Qzf': 1258.0597767539334},
 {u'Af': 3500,
  u'F': 3911.252928814199,
  u'Kz': 0.649098889853901,
  u'Qz': 1878.1526668975748,
  u'Qzf': 1314.7068668283023},
 {u'Af': 3500,
  u'F': 4063.3574552113937,
  u'Kz': 0.674341665256847,
  u'Qz': 1951.192

In [15]:
wind_data['elementArray'].index(a)

0